참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengege Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import matplotlib.patches as patches  # 도형 그리기 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 단면 2차 모멘트<br>Second Moment of Inertia (Area Moment of Inertia)

단면 2차 모멘트 $I$ 는 보의 굽힘 응력 등 공학 계산에서 자주 나타난다.<br>Second moment of inertia $I$ appears frequently in engineering calculations such as bending stress of a beam.

수평 축에 대한 $I$ 는 다음과 같다.<br>The definition of $I$ about a horizontal axis is as follows.

$$
I=\int_A y^2 dA
$$

여기서 $y$는 회전축으로부터의 $dA$ 까지의 수직 거리이다.<br>where $y$ is distance from the rotation axis to $dA$.

### 사각단면의 예<br>A Rectangular Section

#### 도심에 관한 단면 2차 모멘트<br>2nd moment of inertia about centroid

다음과 같이 폭이 $b$ 이고 높이가 $h$ 인 사각 단면의 예를 들어 생각해 보자.<br>Let's think about a rectangular section with base $b$ and height $h$ as follows.

In [ ]:
# Matthias Elsen, Draw rectangles with matplotlib, Python Patterns, 2015, https://matthiaseisen.com/pp/patterns/p0203/
fig1 = plt.figure()
ax1 = fig1.add_subplot(111, aspect='equal')

width = 0.5
height = 1.0

center = np.array((0.5, 0.5))
corner_sw = center + (-0.5) * np.array((width, height))
corner_ne = center + 0.5 * np.array((width, height))

ax1.add_patch(
    patches.Rectangle(
        corner_sw, width, height
    )
)

# https://stackoverflow.com/questions/25761717/matplotlib-simple-and-two-head-arrows
# http://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.annotate.html#matplotlib.axes.Axes.annotate
offset = height * 0.25
plt.annotate(s='$b$', 
             xy=(corner_sw[0], offset + center[1]), 
             xytext=(corner_ne[0], offset + center[1]*0.9625 ),
             arrowprops=dict(arrowstyle='<->')
            )

plt.annotate(s='$h$', 
             xy=(center[0], corner_sw[1]), 
             xytext=(center[0]*0.9625, corner_ne[1]/0.975),
             arrowprops=dict(arrowstyle='<->')
            )

offset_x = width * 0.25
plt.annotate(s='$\\frac{h}{2}$', 
             xy=(offset_x + center[0], center[1]), 
             xytext=(offset_x + center[0]*0.9625, corner_ne[1]/0.955),
             arrowprops=dict(arrowstyle='<->')
            )

overhang_x = width * 0.25
plt.plot((corner_sw[0] - overhang_x, corner_ne[0] + overhang_x), 
         (center[1], center[1]), 'k-.')

# Matplotlib development team, Text properties and layout, matplotlib User's Guide, https://matplotlib.org/users/text_props.html
ax1.set_axis_off()  
plt.show()

이 단면은 상하 대칭으로 도심의 위치는 $\frac{h}{2}$ 이다.<br> 
This section is horizontally symmetric; thus cetroid is at $\frac{h}{2}$.

단면 2차 모멘트의 정의를 다시 한번 생각해보자.<br>
Let's recall the definition of the second moment of inertia.

$$
\begin{align}
I=\int_A y^2 dA
\end{align}
$$

$dA$에 관하여, 미소 면적 $b\cdot dy$를 생각할 수 있다.<br>
Regarding $dA$, one can think about an infinisimal area $b\cdot dy$.

$$
dA=b \cdot dy
$$

도심을 지나는 수평축에 관한 단면 2차 모멘트는 다음과 같다.<br>
Second moment of inertia about a horizontal axis passing the centroid is as follows.

$$
\begin{align}
I&=\int_{y=-\frac{h}{2}}^{y=\frac{h}{2}}y^2\cdot b\cdot dy \\
&=b\int_{y=-\frac{h}{2}}^{y=\frac{h}{2}}y^2\cdot dy \\
&=b \left[ \frac{1}{3}y^3 \right] _{y=-\frac{h}{2}}^{y=\frac{h}{2}} \\
&=\frac{b}{3} \left[ \left( \frac{h}{2} \right)^3 - \left( -\frac{h}{2} \right)^3 \right]\\
&=\frac{b}{3} \left[ \frac{h^3}{8} - \left( - \frac{h^3}{8}\right) \right]\\
&=\frac{b}{3} \frac{h^3}{4} = \frac{bh^3}{12}\\
\end{align} \\
$$

`sympy` 로는 다음과 같이 구할 수 있다.<br>It is as follows in `sympy`.

In [ ]:
b, h, y = sy.symbols('b, h, y', real=True)
I = sy.integrate(b * y**2, (y, -h/2, h/2))

In [ ]:
I

#### $c$ 점 중심의 단면 2차 모멘트와 평행축 정리<br>About coordiante c and Parallel axis theorm

In [ ]:
# Matthias Elsen, Draw rectangles with matplotlib, Python Patterns, 2015, https://matthiaseisen.com/pp/patterns/p0203/
fig1 = plt.figure()
ax1 = fig1.add_subplot(111, aspect='equal')

width = 0.5
height = 1.0

center = np.array((0.5, 0.5))
corner_sw = center + (-0.5) * np.array((width, height))
corner_ne = center + 0.5 * np.array((width, height))

ax1.add_patch(
    patches.Rectangle(
        corner_sw, width, height
    )
)

# https://stackoverflow.com/questions/25761717/matplotlib-simple-and-two-head-arrows
# http://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.annotate.html#matplotlib.axes.Axes.annotate
offset = height * 0.25
plt.annotate(s='$b$', 
             xy=(corner_sw[0], offset + center[1]), 
             xytext=(corner_ne[0], offset + center[1]*0.9625 ),
             arrowprops=dict(arrowstyle='<->')
            )

overhang_x = width * 0.25
offset = height * (-0.25)
plt.plot((corner_sw[0]-overhang_x, corner_ne[0]+overhang_x), 
         (offset + center[1], offset + center[1]), 'k-.')
plt.text(corner_ne[0] + overhang_x + 0.01, offset + center[1] - .01, 'c')

plt.annotate(s='$h$', 
             xy=(center[0], corner_sw[1]), 
             xytext=(center[0]*0.9625, corner_ne[1]/0.975),
             arrowprops=dict(arrowstyle='<->')
            )

offset_x = width * 0.25
plt.annotate(s='$\\frac{h}{2}$', 
             xy=(offset_x + center[0], center[1]), 
             xytext=(offset_x + center[0]*0.9625, corner_ne[1]/0.955),
             arrowprops=dict(arrowstyle='<->')
            )

overhang_x = width * 0.125
plt.plot((corner_sw[0] - overhang_x, corner_ne[0] + overhang_x), 
         (center[1], center[1]), 'k-.')

# Matplotlib development team, Text properties and layout, matplotlib User's Guide, https://matplotlib.org/users/text_props.html
ax1.set_axis_off()  
plt.show()